# Seq2Seq Example. - building Kor-En Translator

## Strategy
- Pretrain Decoder - Endcoder for each dataset (Korean, English)
  - Korean Encoder Training
  - English Decoder Training
  - English Encoder Training
  - Korean Decoder Training
- End-to-End Training - parallel corpus text dataset
  - Korean - English Translator
  - English - Korean Translator
- 


In [ ]:
from tokenizers.pre_tokenizers import PreTokenizer
import mecab_ko

class MecabKoPretokenizer(PreTokenizer):
    def __init__(self) -> None:
        pass
    
    def pre_tokenize(self, pretok):
        return super().pre_tokenize(pretok)
    
    def pre_tokenize_str(self, sequence):
        return super().pre_tokenize_str(sequence)

In [15]:
from data import get_default_tokenizer
from transformers import AutoTokenizer
from tokenizers import pre_tokenizers,Regex
# mecab_tokenizer = AutoTokenizer.from_pretrained('Jinhwan/krelectra-base-mecab')

ko_tokenizer = get_default_tokenizer('ko')
tokens = ko_tokenizer.encode("어머니가 방에 들어가신다.")

# mtokens = mecab_tokenizer.encode("어머니가 방에 들어가신다.")

print(ko_tokenizer.decode(tokens))


# print(mecab_tokenizer.decode(mtokens))


어 머 니 가   방 에   들 어 가 신 다.


In [8]:
from model import GRUEncoder, get_default_tokenizer
from data import KoEnParallel, Korean, English, Wikipedia
from transformers import AutoTokenizer, GPT2TokenizerFast
from torch.utils import data

# ko_tokenizer = AutoTokenizer.from_pretrained("skt/ko-gpt-trinity-1.2B-v0.5")
tokenizer = get_default_tokenizer("en")
data = Wikipedia()
print(tokenizer.decode(tokenizer.pad_token_id))

ko_dataset = English(tokenizer, 1024, 'train')
dataloader = data.DataLoader(ko_dataset, batch_size=10, collate_fn=ko_dataset)
ko_encoder = GRUEncoder(len(tokenizer.vocab), 1024, 1024, 2, padding_id=tokenizer.pad_token_id)

next(iter(dataloader))

ImportError: cannot import name 'GRUEncoder' from 'model' (/home/fritzprix/my_work/ml_note/labs/seq2seq_trans/model/__init__.py)

In [4]:
import pytorch_lightning as pl
from pytorch_lightning import loggers, callbacks

logger = loggers.TensorBoardLogger('logs')
model_checkout = callbacks.ModelCheckpoint('model/encoder', filename='model-{epoch}-{val_loss:.3f}')

trainer = pl.Trainer(logger=logger, accelerator='gpu', callbacks=[model_checkout])
trainer.fit(model=ko_encoder, train_dataloaders=dataloader, val_dataloaders=dataloader)




GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/fritzprix/miniconda3/envs/directml/lib/python3.10/site-packages/pytorch_lightning/loops/utilities.py:94: PossibleUserWarning: `max_epochs` was not set. Setting it to 1000 epochs. To train without an epoch limit, set `max_epochs=-1`.
  rank_zero_warn(
Missing logger folder: logs/lightning_logs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/home/fritzprix/miniconda3/envs/directml/lib/python3.10/site-packages/pytorch_lightning/utilities/model_summary/model_summary.py:411: UserWarning: A layer with UninitializedParameter was found. Thus, the total number of parameters detected may be inaccurate.
  warning_cache.warn(

  | Name      | Type             | Params
-----------------------------------------------
0 | embedding | Embedding        | 52.4 M
1 | enc       | GRU              | 31.5 M
2 | fc        | LazyLinear       | 0     
3 | loss

Sanity Checking: 0it [00:00, ?it/s]

/home/fritzprix/miniconda3/envs/directml/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:224: PossibleUserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 16 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(


IndexError: too many indices for tensor of dimension 2

In [7]:
from transformers import PreTrainedTokenizer, PreTrainedTokenizerFast
from data import get_default_tokenizer
tokenizer = get_default_tokenizer('en')
enc = tokenizer.encode("하늘에 계신 우리 아버지 아버지의 이름이 거룩히 빛나시며 ")
tokenizer.convert_tokens_to_string(enc)

Using custom data configuration Moo--korean-parallel-corpora-152c18962d41c571
Found cached dataset csv (/home/fritzprix/.cache/huggingface/datasets/Moo___csv/Moo--korean-parallel-corpora-152c18962d41c571/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
from datasets import Dataset

from_data = Dataset.from_dict({"v": ["afeo.jfaew.fjaw.lifja.ilef.jli", "afeo.jfaew.fjaw.lifja.ilef.jli"]})

to_data= from_data.map(lambda x: {"vv": x.split('.')}, input_columns='v', remove_columns=['v'])


# data = Wikipedia()

  0%|          | 0/2 [00:00<?, ?ex/s]

In [1]:
from model import EncoderDecoder
from torch.utils import data
from torchtext import functional as TF
from model import EncoderDecoder
from data import get_default_tokenizer, KoEnParallel, ParallelCollator

ko_tokenizer = get_default_tokenizer('ko')
en_tokenizer = get_default_tokenizer('en')


collator = ParallelCollator(from_tokenizer=ko_tokenizer, 
                            to_tokenizer=en_tokenizer, 
                            from_pad_id=ko_tokenizer.encode('<pad>')[0], 
                            to_pad_id=en_tokenizer.encode('<pad>')[0], 
                            from_eos_id=ko_tokenizer.encode('<eos>')[0], 
                            to_bos_id=en_tokenizer.encode('<bos>')[0], 
                            to_eos_id=en_tokenizer.encode('<eos>')[0])

para_dataset = KoEnParallel(split='train')
train_dataloader = data.DataLoader(dataset=para_dataset, collate_fn=collator, batch_size=10)
seq2seq_trans = EncoderDecoder()

seq2seq_trans.training_step(next(iter(train_dataloader)))

Using custom data configuration Moo--korean-parallel-corpora-152c18962d41c571
Found cached dataset csv (/home/fritzprix/.cache/huggingface/datasets/Moo___csv/Moo--korean-parallel-corpora-152c18962d41c571/0.0.0/6b34fb8fcf56f7c8ba51dc895bfa2bfbe43546f190a60fcf74bb5e8afdcc2317)


  0%|          | 0/3 [00:00<?, ?it/s]

/home/fritzprix/miniconda3/envs/directml/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


torch.Size([10, 55])
torch.Size([10, 56])
torch.Size([55, 10, 30000]) vs . torch.Size([10, 56])


tensor(10.3964, grad_fn=<NllLossBackward0>)

In [1]:
from model import EncoderDecoder

seq2seq_trans = EncoderDecoder()


/home/fritzprix/miniconda3/envs/directml/lib/python3.10/site-packages/torch/nn/modules/lazy.py:180: UserWarning: Lazy modules are a new feature under heavy development so changes to the API or functionality can happen at any moment.
  warnings.warn('Lazy modules are a new feature under heavy development '


In [29]:
import numpy as np

def prediction(X, theta):
    """다중 선형 회귀 가정 함수. 모든 데이터에 대한 예측 값을 numpy 배열로 리턴한다"""
    # 지난 실습의 코드를 여기에 붙여 넣으세요
    return X@theta
    

def gradient_descent(X, theta, y, iterations, alpha):
    """다중 선형 회귀 경사 하강법을 구현한 함수"""
    m = len(X)  # 입력 변수 개수 저장
    
    for _ in range(iterations):
        # 여기에 코드를 작성하세요
        y_hat = prediction(X, theta)
        error = np.square(y_hat - y).mean()
        theta = theta - (error * X).mean() * alpha
        
    return theta
    

# 입력 변수
house_size = np.array([1.0, 1.5, 1.8, 5, 2.0, 2.5, 3.0, 3.5, 4.0, 5.0, 6.0, 7.0, 8.0, 8.5, 9.0, 10.0])  # 집 크기
distance_from_station = np.array([5, 4.6, 4.2, 3.9, 3.9, 3.6, 3.5, 3.4, 2.9, 2.8, 2.7, 2.3, 2.0, 1.8, 1.5, 1.0])  # 지하철역으로부터의 거리 (km)
number_of_rooms = np.array([1, 1, 1, 1, 2, 2, 2, 2, 3, 3, 3, 3, 4, 4, 4, 4])  # 방 수

# 목표 변수
house_price = np.array([3, 3.2, 3.6 , 8, 3.4, 4.5, 5, 5.8, 6, 6.5, 9, 9, 10, 12, 13, 15])  # 집 가격

# 설계 행렬 X 정의
X = np.array([
    np.ones(16),
    house_size,
    distance_from_station,
    number_of_rooms
]).T

# 입력 변수 y 정의
y = house_price

# 파라미터 theta 초기화
theta = np.array([0, 0, 0, 0])

# 학습률 0.01로 100번 경사 하강
theta = gradient_descent(X, theta, y, 100, 0.01)

theta


[  9.    10.24  12.96  64.    11.56  20.25  25.    33.64  36.    42.25
  81.    81.   100.   144.   169.   225.  ]
66.55625
[ 331.83805439  346.20660328  354.05776158  825.54942309  413.23856953
  475.61855699  532.20191811  606.65281291  714.61991961  837.76806927
 1099.32504196 1176.30497016 1484.62196917 1689.32576672 1804.70493675
 2064.14343319]
922.2611129193972
[ 52472.31795003  53867.72891508  52747.55999355  99867.47338991
  64974.55098924  68462.17226694  74778.28724605  81544.28936522
  98607.40077561 115566.57712244 135338.29660135 148100.79766009
 188250.28635679 197452.03865028 203407.73447348 218216.73310564]
115853.39030385722
[7.13747313e+08 7.31711310e+08 7.13779373e+08 1.32528982e+09
 8.83377538e+08 9.23588208e+08 1.00658645e+09 1.09317513e+09
 1.32514421e+09 1.55301004e+09 1.79911421e+09 1.97308711e+09
 2.50970623e+09 2.61128232e+09 2.68008463e+09 2.85595111e+09]
1543664687.3721611
[1.24571624e+17 1.27705379e+17 1.24571624e+17 2.31255545e+17
 1.54176849e+17 1.611840

/tmp/ipykernel_887/2366012682.py:16: RuntimeWarning: overflow encountered in square
  error = np.square(y_hat - y).mean()
/tmp/ipykernel_887/2366012682.py:17: RuntimeWarning: overflow encountered in square
  print(np.square(y_hat - y))


array([-inf, -inf, -inf, -inf])